## Install and import library


In [1]:
!pip install git+https://github.com/DangLeUyen/DPEImputation.git

  Cloning https://github.com/DangLeUyen/DPEImputation.git to /tmp/pip-req-build-jjffrk49
  Running command git clone --filter=blob:none --quiet https://github.com/DangLeUyen/DPEImputation.git /tmp/pip-req-build-jjffrk49
  Resolved https://github.com/DangLeUyen/DPEImputation.git to commit cdffc3cb76d8e4eda13bae0aa6b821bc20bbeeab
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 497.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 22.9 MB/s eta 0:00:00
  Created wheel for DPEImputation: filename=DPEImputation-0.1.0-py3-none-any.whl size=6185 sha256=6e3c0cc3cee53b13f5f88a738f4a1223fc04dc15a7474ee4ec1238d8a5587479
  Stored in directory: /tmp/pip-ephem-wheel-cache-6f0teykd/wheels/ea/b7/79/1faeb14dc67afa9acbaa5079fd4e482ee9c17a596e9ba0f412
Successfully built DPEImputation
  Attempting uninstall: numpy
    Found existing installation: nump

In [2]:
import numpy as np
import time
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from DPEImputation import DPEImputer

In [3]:
def generate_randomly_missing(X , missing_rate):
    """
    Creates a randomly missing mask for the input data.

    Args:
        data (np.ndarray): The input data.
        missing_rate (float): The ratio of missing values to create.

    Returns:
        np.ndarray: An array with the same shape as `data` where missing values are marked as NaN.
    """
    # Create a copy of the input array to avoid modifying the original data
    X_copy=np.copy(X)
    # Calculate the total number of elements in the array
    total_elements = X_copy.size
    # Determine the number of elements to be replaced with NaN
    num_missing = round(missing_rate * total_elements)
    # Generate random indices where NaN will be introduced
    missing_indices = np.random.randint(0, total_elements, num_missing)
    # Flatten the array to apply NaN values
    X_copy_flat = X_copy.flatten()
    # Assign NaN to the selected indices
    X_copy_flat[missing_indices] = np.nan
    # Reshape the array back to its original shape
    X_nan = X_copy_flat.reshape(X_copy.shape)

    return X_nan

def normalize_data(X):
  scaler = StandardScaler()
  scaler.fit(X)
  return scaler.transform(X)

### Generating dataset with label

In [11]:
#create a sample data
X1 = np.random.randint(0, 100, size=(1000, 50)).astype('float64')
# Generate random labels (e.g., binary classification)
y = np.random.randint(0, 2, size=X1.shape[0])

X1 = normalize_data(X1)
#create missingness on data
missing_rate = 0.5
missing_X1 = generate_randomly_missing(X1, missing_rate)


In [12]:
imputer1 = DPEImputer()
start = time.time()

X1_imputed = imputer1.fit(missing_X1, y).transform(missing_X1, y)

rmse1 = mean_squared_error(X1, X1_imputed)
duration1 = time.time() - start
print("Imputation done after: {} (seconds) and have Rmse = {}".format(duration1, rmse1))

Imputation done after: 19.387824296951294 (seconds) and have Rmse = 0.4060268734865917


### Generating dataset without label

In [6]:
#create a sample data
X2 = np.random.randint(0, 100, size=(1000, 50)).astype('float64')

#create missingness on data
X2 = normalize_data(X2)
missing_rate = 0.5
missing_X2 = generate_randomly_missing(X2, missing_rate)


In [7]:
imputer2 = DPEImputer()
start = time.time()
X2_imputed = imputer2.fit_transform(missing_X2)
rmse2 = mean_squared_error(X2, X2_imputed)
duration2 = time.time() - start
print("Imputation done after: {} (seconds) and have Rmse2 = {}".format(duration2, rmse2))

Imputation done after: 15.959425210952759 (seconds) and have Rmse2 = 0.3973782389663099


### Using digits dataset (with label)

In [8]:
from sklearn.datasets import load_digits

# Tải dữ liệu digits
digits = load_digits()

# Lấy dữ liệu đặc trưng và nhãn
X = digits.data
y = digits.target
print(X.shape)
rmid = np.where(sum(X!=0)<10)
X = np.delete(X, rmid,axis = 1)
print(X.shape)
X = normalize_data(X)

(1797, 64)
(1797, 54)


In [9]:
digits_missing_data = generate_randomly_missing(X , missing_rate)

start = time.time()
dpei = DPEImputer()
digits_imputed = dpei.fit(digits_missing_data, y, window_size=5).transform(digits_missing_data, y)
rmse3 = mean_squared_error(X, digits_imputed)
duration3 = time.time() - start
print("Imputation done after: {} (seconds) and have Rmse = {}".format(duration3, rmse3))

Imputation done after: 26.447102069854736 (seconds) and have Rmse = 0.20217014812196196
